In [1]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Flatten
import pylidc as pl
from unet import UNet

Using TensorFlow backend.


In [2]:
scans = pl.query(pl.Scan)

poke poke

In [3]:
scan = scans[0]

In [4]:
images = scan.load_all_dicom_images(verbose=False)
i = images[0]
i.pixel_array, i.pixel_array.shape

(array([[-2048, -2048, -2048, ..., -2048, -2048, -2048],
        [-2048, -2048, -2048, ..., -2048, -2048, -2048],
        [-2048, -2048, -2048, ..., -2048, -2048, -2048],
        ..., 
        [-2048, -2048, -2048, ..., -2048, -2048, -2048],
        [-2048, -2048, -2048, ..., -2048, -2048, -2048],
        [-2048, -2048, -2048, ..., -2048, -2048, -2048]], dtype=int16),
 (512, 512))

In [3]:
def get_annotated_slices(ann):
    zs = ann.bbox()[-1]
    return zs.start, zs.stop

In [9]:
set.union(*(set(range(*get_annotated_slices(ann))) for ann in scan.annotations))

{23,
 24,
 25,
 26,
 27,
 28,
 44,
 45,
 46,
 47,
 48,
 49,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71}

ok, real stuff now

In [4]:
model = Sequential([UNet((512, 512, 1)), Flatten(), Dense(1, activation='sigmoid')])

In [5]:
model.compile('adam', loss='binary_crossentropy')

In [6]:
def get_data(indices):
    X, y = [], []
    for i in indices:
        scan = scans[i]
        images = scan.load_all_dicom_images(verbose=False)
        X.extend((im.pixel_array for im in images))
        nodule_zs = set.union(*(set(range(*get_annotated_slices(ann))) for ann in scan.annotations))
        y.extend((i in nodule_zs for i in range(len(images))))
    X = np.array(X)
    return X.reshape(*X.shape, 1), np.array(y).reshape(-1, 1)

In [7]:
X_train, y_train = get_data(range(3))

In [ ]:
model.fit(X_train, y_train, batch_size=1, epochs=10)

Epoch 1/10
 45/336 [===>..........................] - ETA: 6:00 - loss: 0.7492